### In this notebook

- Generating a list of all human protein-coding genes (optional).
- Reading a list of gene sets defined based on GO annotations.
- Reading the expression data.
- Performing gene set enrichment analysis using the XL-mHG test.

### Versions of Python packages used to generate these results

In [1]:
import genometools
print('genometools:', genometools.__version__)

genometools: 2.0.3


### 1. Generating a list of all human-protein-coding genes (you can skip this if you want)

When performing gene set enrichment analysis, it is important to define your "universe" of genes. We are restricting ourselves to protein-coding genes. Next, there are different databases for obtaining a list of protein-coding genes. Here, we're using genome annotations from [Ensembl](http://useast.ensembl.org/info/data/ftp/index.html) (release 83). This file is relatively large, and we're downloading it from the Ensembl FTP site.

Note: If you don't want to deal with generating this file for now, skip to the next section! The output file (`protein_coding_genes_human_ensembl83.tsv`) is already included with these notebooks, in the `data` directory.

In [2]:
import os
    
from genometools import misc

ensembl_annotation_url = 'ftp://ftp.ensembl.org/pub/release-83/gtf/homo_sapiens/Homo_sapiens.GRCh38.83.gtf.gz'
ensembl_annotation_file = os.path.join('..', 'output', 'Homo_sapiens.GRCh38.83.gtf.gz')

# download the Ensembl gene annotation file
misc.ftp_download(ensembl_annotation_url, ensembl_annotation_file)

Now, we're extracting a list of all protein-coding genes from the Ensembl gene annotation file. We're using a script (`ensembl_extract_protein_coding_genes.py`) from the *genometools* package for this. In order to make a command-line call from inside a jupyter notebook, we have to use an exclamation mark (e.g., `!echo "Hello, world"`). We're also using the values of some (Python) variables to create the command, using the "$" sign. Don't worry if this doesn't make sense to you - the following sections do not involve any command-line calls.

In [3]:
import os

genome_file = os.path.join('..', 'output', 'protein_coding_genes_human_ensembl83.tsv')

!ensembl_extract_protein_coding_genes.py -a "$ensembl_annotation_file" -o "$genome_file"

[2016-10-27 11:38:19] INFO: Regular expression used for filtering chromosome names: "(?:\d\d?|MT|X|Y)$"
[2016-10-27 11:38:19] INFO: Parsing data...
[2016-10-27 11:38:45] INFO: done (parsed 2569155 lines).
[2016-10-27 11:38:45] INFO: 
[2016-10-27 11:38:45] INFO: Gene chromosomes (25):
[2016-10-27 11:38:45] INFO: 	1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 2, 20, 21, 22, 3, 4, 5, 6, 7, 8, 9, MT, X, Y
[2016-10-27 11:38:45] INFO: 
[2016-10-27 11:38:45] INFO: Excluded chromosomes (15):
[2016-10-27 11:38:45] INFO: 	GL000009.2, GL000194.1, GL000195.1, GL000205.2, GL000213.1, GL000218.1, GL000219.1, KI270711.1, KI270713.1, KI270721.1, KI270726.1, KI270727.1, KI270728.1, KI270731.1, KI270734.1
[2016-10-27 11:38:45] INFO: 
[2016-10-27 11:38:45] INFO: Gene sources:
[2016-10-27 11:38:45] INFO: 	ensembl_havana: 18825
[2016-10-27 11:38:45] INFO: 	havana: 791
[2016-10-27 11:38:45] INFO: 	ensembl: 226
[2016-10-27 11:38:45] INFO: 	insdc: 13
[2016-10-27 11:38:45] INFO: 
[2016-10-27 11:38:45] INFO: Gene 

### 2. Reading a list of gene sets, defined based on Gene Ontology annotations

The *genometools* package provides a class, `GeneSetCollection`, that represents a list of gene sets (which are in turn represented as objects of the `GeneSet` class). This class allows reading ("importing") and writing ("exporting") gene sets as plain-text files. Here, we are using the `GeneSetCollection.read_tsv()` function to import 7,000 or so gene sets defined based on GO terms.

More specifically, each gene set contains all protein-coding genes in the human genome annotated with a particular GO term. Furthermore, in generating this file, only annotations that were made based on experimental evidence were considered. (The majority of GO annotations are based on computational predictions, and are presumably less accurate.) Also, GO terms that had less than 5 or more than 200 genes annotated with them were excluded, since those terms are too specific or too general, respectively.

In [4]:
import os

from genometools.basic import GeneSetCollection

gene_set_file = os.path.join('..', 'data', 'GO_gene_sets_human_ensembl83_goa153_ontology2016-01-18.tsv')
print('Gene set file:', gene_set_file)

gene_sets = GeneSetCollection.read_tsv(gene_set_file)
print(gene_sets)

Gene set file: ../data/GO_gene_sets_human_ensembl83_goa153_ontology2016-01-18.tsv
<GeneSetCollection object (n=6921)>


### 3. Reading the expression data

In this example, we will compare gene expression levels between tumors from two sets of patients diagnosed with breast cancer (specifically, invasive ductal carcinoma) from the TCGA project. The first set of patients are those that died within 5 years of their diagnosis. The second set of patients survived for at least 5 years.

Note: All patients in this example were younger than 60 years at the time of diagnosis. I chose to filter patients using this cutoff to make it less likely that the death of a patient was caused by factors unrelated to their breast cancer diagnosis. (I did not find information on the cause of death in the TCGA data.)

In [5]:
import os

from genometools.expression import ExpMatrix

dead_expression_file = os.path.join('..', 'data', 'brca_expression_5yr_dead.tsv')
survive_expression_file = os.path.join('..', 'data', 'brca_expression_5yr_survive.tsv')

matrix_dead = ExpMatrix.read_tsv(dead_expression_file)
matrix_survive = ExpMatrix.read_tsv(survive_expression_file)

### 4. Performing the enrichment analysis using the XL-mHG test

We are now almost ready to get started with the gene set enrichment analysis itself. We need to read the list of protein-coding genes. Similarly to the gene sets, the *genometools* package provides a data structure for that, `ExpGenome`.

In [6]:
import os

from genometools.expression import ExpGenome

genome_file = os.path.join('..', 'data', 'protein_coding_genes_human_ensembl83.tsv')

genome = ExpGenome.read_tsv(genome_file)
print(genome)

<ExpGenome object with 19808 genes>


Finally, we are ready to run our first gene set enrichment analysis in Python.

In [7]:
from genometools.enrichment import GeneSetEnrichmentAnalysis

# The significance threshold will be adjusted for multiple testing using the Bonferroni method
pval_thresh = 0.05

diff = matrix_dead.median(axis=1) - matrix_survive.median(axis=1)
diff.sort_values(ascending=False, inplace=True)

enrichment = GeneSetEnrichmentAnalysis(genome, gene_set_coll)
enriched = enrichment.get_rank_based_enrichment(diff.index, pval_thresh=pval_thresh)

# sort significantly enriched GO terms by their E-score (higher E-score = stronger enrichment)
enriched = sorted(enriched, key=lambda x:-x.escore)

# print nicely formatted list of enriched GO terms
for i, enr in enumerate(enriched):
    print('%02d. %s' %(i, enr.get_pretty_format()))

[2016-10-27 11:38:47] INFO: Starting new HTTPS connection (1): api.plot.ly
[2016-10-27 11:38:49] INFO: Generating gene-by-gene set membership matrix...
[2016-10-27 11:38:49] WARNING: No E-score p-value threshold supplied. The E-score p-value threshold will be set to theglobal significance threshold. This will result in conservative E-scores.
[2016-10-27 11:38:50] INFO: Conducting 6921 tests.
[2016-10-27 11:38:50] INFO: Using Bonferroni-corrected p-value threshold: 7.2e-06
[2016-10-27 11:38:55] INFO: 217 / 6921 gene sets were found to be significantly enriched (p-value <= 7.2e-06).
00. condensed chromosome outer kinetochore (12 / 12 @ 3908), p=2.5e-08, e=30.9x
01. regulation of attachment of spindle microtubules to kinetochore (5 / 11 @ 371), p=5.1e-06, e=24.2x
02. mitotic chromosome condensation (7 / 11 @ 535), p=2.0e-08, e=23.5x
03. condensed chromosome kinetochore (14 / 23 @ 1422), p=5.3e-10, e=18.8x
04. condensed chromosome, centromeric region (16 / 27 @ 1422), p=4.2e-11, e=18.3x
05

In [8]:
# examine the most strongly enriched GO term
enr = enriched[0]

print('Gene set:', enr.gene_set)

print('Number of genes annotated with GO term above the mHG cutoff:', enr.k)

print('Gene names:', enr.genes_above_cutoff)

# print the indices (0-based ranks) of the genes above the cutoff
print('Gene indices in ranked list:', enr.indices[:enr.k])

Gene set: <GeneSet "condensed chromosome outer kinetochore" (id=GO:0000940, source="GO", collection="CC", size=12
Number of genes annotated with GO term above the mHG cutoff: 12
Gene names: ['PLK1', 'BUB1B', 'CENPF', 'SKA3', 'BUB1', 'CCNB1', 'CENPE', 'SPDL1', 'NUP133', 'NDC80', 'SKA2', 'SKA1']
Gene indices in ranked list: [  20  101  191  306  318  319 1222 1333 1406 2319 3753 3907]


### Copyright and License

Copyright (c) 2016 Florian Wagner.

This work is licensed under a [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License](http://creativecommons.org/licenses/by-nc-sa/4.0/).